Web parsing with Python, Beautiful Soup and Selenium
1. Get any HTML
It's very easy to extract the source code of a web page in Python


In [1]:

from bs4 import BeautifulSoup, SoupStrainer
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup, SoupStrainer
import requests
import json
import pandas as pd
from IPython.display import IFrame



2. Scrape Airbnb page
Let's get to the website and look for some apartments


In [2]:
# Let's plan a trip to Austrian Alps
airbnb_url = 'https://www.airbnb.com/s/Mayrhofen--Austria/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&date_picker_type=calendar&query=Mayrhofen%2C%20Austria&place_id=ChIJbzLYLzjdd0cRDtGuTzM_vt4&checkin=2021-02-06&checkout=2021-02-13&adults=4&source=structured_search_input_header&search_type=autocomplete_click'
soup = BeautifulSoup(requests.get(airbnb_url).content, 'html.parser')
print(soup.prettify())
listings[0].get_text()

listings[0].find_all('a')[0].get('href')


<!DOCTYPE html>
<html data-is-hyperloop="true" dir="ltr" lang="en">
 <meta charset="utf-8"/>
 <meta content="en" name="locale"/>
 <meta content="notranslate" name="google"/>
 <meta content="authenticity_token" id="csrf-param-meta-tag" name="csrf-param"/>
 <meta content="" id="csrf-token-meta-tag" name="csrf-token"/>
 <meta content="" id="english-canonical-url"/>
 <meta content="on" name="twitter:widgets:csp"/>
 <meta content="yes" name="mobile-web-app-capable"/>
 <meta content="yes" name="apple-mobile-web-app-capable"/>
 <meta content="Airbnb" name="application-name"/>
 <meta content="Airbnb" name="apple-mobile-web-app-title"/>
 <meta content="#ffffff" name="theme-color"/>
 <meta content="#ffffff" name="msapplication-navbutton-color"/>
 <meta content="black-translucent" name="apple-mobile-web-app-status-bar-style"/>
 <meta content="/?utm_source=homescreen" name="msapplication-starturl"/>
 <script>
  (function() {
  var pgRequest = new XMLHttpRequest();
  var diffStamp = Date.now().toSt

NameError: name 'listings' is not defined

3. Inspect all data elements on search page
4. Write a scraping function


In [3]:
listings = soup.find_all('div', '_gig1e7')

In [4]:
listings[0]

<div class="_gig1e7"><div class="cd239om dir dir-ltr"><div class="ch1c6qs dir dir-ltr" style="--transition-element_transition-delay:0ms"><div class="_11ry7lz" data-testid="shimmer-legacy-listing-section-item"><div style="margin-top:12px;margin-bottom:24px"><div class="_7qp4lh"></div></div><div class="c1k4n2ms dir dir-ltr"><div class="mertcch dir dir-ltr"><span aria-busy="true" style="display:block;height:100%;width:100%;background-color:white;border-radius:var(--i-g-gvoq)"><span class="_8kplbn" style="animation-play-state:running;height:100%;width:100%;border-radius:var(--i-g-gvoq)"></span></span></div><div class="iabnva0 dir dir-ltr"><div style="margin-bottom:4px"><span aria-busy="true" style="display:block;height:18px;width:40%;background-color:white;border-radius:10px"><span class="_8kplbn" style="animation-play-state:running;height:100%;width:100%;border-radius:10px"></span></span></div><span aria-busy="true" style="display:block;height:24px;width:90%;background-color:white;border-

In [5]:
listings[0].get_text()

'Entire rental unit in MayrhofenRosa (MHO134) for 5 persons.5 guests · 2 bedrooms · 3 beds · 1 bathWifi · Kitchen4.67\xa0(3 reviews)$17\xa0/ night$17 per night'

In [6]:
listings[0].find_all('a')[0].get('href')

'/rooms/17485726?adults=4&previous_page_section_name=1000&federated_search_id=dd33ec9d-eee1-46bd-b7b2-6ad6566b590e'

In [7]:
# First Generation :)
def extract_basic_features(listings_html):
    features_dict = {}
    
    url = listings_html.find('a').get('href')
    #name = listings_html.find("div", {"class": "_hxt6u1e"}).get('aria-label')
    number = listings_html.find('div',{'class': 'i1wgresd'}).get_text()
    description = listings_html.find("div", {"class": "c1bx80b8"}).get_text()
    
    features_dict['url'] = url
    #features_dict['name'] = name
    features_dict['number'] =number
    features_dict['description'] = description
    
    return features_dict


In [8]:
extract_basic_features(listings[0])

{'url': '/rooms/17485726?adults=4&previous_page_section_name=1000&federated_search_id=dd33ec9d-eee1-46bd-b7b2-6ad6566b590e',
 'number': '5 guests · 2 bedrooms · 3 beds · 1 bath',
 'description': 'Rosa (MHO134) for 5 persons.'}

In [9]:
# Second Generation :)
def extract_basic_features(listing_html):
    features_dict = {}
    
    try:
        url = listing_html.find('b').get('href')
    except:
        url = 'empty'
    try:
        name = listing_html.find("div", {"class": "_hxt6u1e"}).get('aria-label')
    except:
        name = 'empty'
    try:
        header = listing_html.find("div", {"class": "_b14dlit"}).text
    except:
        header = 'empty'
    
    
    features_dict['url'] = url
    features_dict['name'] = name
    features_dict['header'] = header
    
    return features_dict


In [10]:
extract_basic_features(listings[15])

{'url': 'empty', 'name': 'empty', 'header': 'empty'}

In [11]:
# too many separate extractions
RULES_SEARCH_PAGE = {
    #'url': {'tag': 'a', 'get': 'href'},
    #'name': {'tag': 'div', 'class': '_hxt6u1e', 'get': 'aria-label'},
    #'header': {'tag': 'div', 'class': 'o1n80548'},
    'rooms': {'tag': 'span', 'class': 'mp2hv9t'},
    'facilities': {'tag': 'div', 'class': 'i1wgresd'},
    'badge': {'tag': 'div', 'class': 'c1bx80b8'},
    'rating_n_reviews': {'tag': 'span', 'class': 'rpz7y38'},
    'price': {'tag': 'span', 'class': 'a8jt5op'},
   
}

In [12]:
# Third Generation :)
def extract_element(listing_html, params):
    # 1. Find the right tag
    if 'class' in params:
        elements_found = listing_html.find_all(params['tag'], params['class'])
    else:
        elements_found = listing_html.find_all(params['tag'])

    # 2. Extract the right element
    tag_order = params.get('order', 0)
    element = elements_found[tag_order]
        
    # 3. Get text
    if 'get' in params:
        output = element.get(params['get'])
    else:
        output = element.get_text()

    return output

Explore pagination

In [13]:
airbnb_url

'https://www.airbnb.com/s/Mayrhofen--Austria/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&date_picker_type=calendar&query=Mayrhofen%2C%20Austria&place_id=ChIJbzLYLzjdd0cRDtGuTzM_vt4&checkin=2021-02-06&checkout=2021-02-13&adults=4&source=structured_search_input_header&search_type=autocomplete_click'

In [14]:
def get_listings(search_page):
    soup = BeautifulSoup(requests.get(search_page).content, 'html.parser')
    listings = soup.find_all('div', '_gig1e7')

    return listings


In [15]:
# it works
len(get_listings(airbnb_url))


20

In [16]:
# let's try next page
new_url = airbnb_url + '&items_offset=20'
len(get_listings(new_url))


20

 Collect all urls

In [17]:
# let's iterate through all 15 pages
all_listings = []
for i in range(15):
    offset = 20 * i
    new_url = airbnb_url + f'&items_offset={offset}'
    new_listings = get_listings(new_url)
    all_listings.extend(new_listings)
    
    # let's check if it's scraping
    print(len(all_listings))

20
40
60
80
100
120
140
160
180
181
181
181
181
181
181


10. Scrape all search pages
    1.build all urls
    2.iteratively scrape them

In [18]:
# 1. build all urls
def build_urls(main_url, listings_per_page=20, pages_per_location=15):
    url_list = []
    for i in range(pages_per_location):
        offset = listings_per_page * i
        url_pagination = main_url + f'&items_offset={offset}'
        url_list.append(url_pagination)
    
    return url_list


In [19]:
# safe function to extract all features from one page
def extract_page_features(soup, rules):
    features_dict = {}
    for feature in rules:
        try:
            features_dict[feature] = extract_element(soup, rules[feature])
        except:
            features_dict[feature] = 'empty'
    
    return features_dict

In [20]:
# 2. Iteratively scrape pages
def process_search_pages(url_list):
    features_list = []
    for page in url_list:
        listings = get_listings(page)
        for listing in listings:
            features = extract_page_features(listing, RULES_SEARCH_PAGE)
            features_list.append(features)

    return features_list


In [21]:
# build a list of URLs
url_list = build_urls(airbnb_url)
url_list

['https://www.airbnb.com/s/Mayrhofen--Austria/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&date_picker_type=calendar&query=Mayrhofen%2C%20Austria&place_id=ChIJbzLYLzjdd0cRDtGuTzM_vt4&checkin=2021-02-06&checkout=2021-02-13&adults=4&source=structured_search_input_header&search_type=autocomplete_click&items_offset=0',
 'https://www.airbnb.com/s/Mayrhofen--Austria/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&date_picker_type=calendar&query=Mayrhofen%2C%20Austria&place_id=ChIJbzLYLzjdd0cRDtGuTzM_vt4&checkin=2021-02-06&checkout=2021-02-13&adults=4&source=structured_search_input_header&search_type=autocomplete_click&items_offset=20',
 'https://www.airbnb.com/s/Mayrhofen--Austria/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&date_picker_type=calendar&query=Mayrhofen%2C%20Austria&place_id=ChIJbzLYLzjdd0cRDtGuTzM_vt4&checkin=2021-02-06&checkout=2021-02-13&adults=4&source=structured_search_input_header&search_type=autocomplete_click&items_offset=40',
 'https://www.airbnb

In [22]:
# try for one page
base_features = process_search_pages(url_list[:1])
base_features

[{'rooms': '5 guests',
  'facilities': '5 guests · 2 bedrooms · 3 beds · 1 bath',
  'badge': 'Rosa (MHO134) for 5 persons.',
  'rating_n_reviews': '4.67',
  'price': '$17 per night'},
 {'rooms': '5 guests',
  'facilities': '5 guests · 2 bedrooms · 3 beds · 2 baths',
  'badge': 'Eberharter (MHO153) for 5 persons.',
  'rating_n_reviews': '4.86',
  'price': '$17 per night'},
 {'rooms': '5 guests',
  'facilities': '5 guests · 2 bedrooms · 4 beds · 1 bath',
  'badge': 'Sonnenheim for 5 persons.',
  'rating_n_reviews': '4.25',
  'price': '$17 per night'},
 {'rooms': '4 guests',
  'facilities': '4 guests · 1 bedroom · 2 beds · 1 bath',
  'badge': 'Gredler (MHO250) for 4 persons.',
  'rating_n_reviews': '4.40',
  'price': '$17 per night'},
 {'rooms': '5 guests',
  'facilities': '5 guests · 2 bedrooms · 3 beds · 2 baths',
  'badge': 'Georg (MHO280) for 5 persons.',
  'rating_n_reviews': 'empty',
  'price': '$17 per night'},
 {'rooms': '4 guests',
  'facilities': '4 guests · 1 bedroom · 2 beds ·

In [23]:
df = pd.DataFrame.from_dict(base_features)

In [24]:
df

,rooms,facilities,badge,rating_n_reviews,price
0,5 guests,5 guests · 2 bedrooms · 3 beds · 1 bath,Rosa (MHO134) for 5 persons.,4.67,$17 per night
1,5 guests,5 guests · 2 bedrooms · 3 beds · 2 baths,Eberharter (MHO153) for 5 persons.,4.86,$17 per night
2,5 guests,5 guests · 2 bedrooms · 4 beds · 1 bath,Sonnenheim for 5 persons.,4.25,$17 per night
3,4 guests,4 guests · 1 bedroom · 2 beds · 1 bath,Gredler (MHO250) for 4 persons.,4.40,$17 per night
4,5 guests,5 guests · 2 bedrooms · 3 beds · 2 baths,Georg (MHO280) for 5 persons.,empty,$17 per night
5,4 guests,4 guests · 1 bedroom · 2 beds · 1 bath,Wegscheider (MHO328) for 4 persons.,empty,$17 per night
6,10 guests,10 guests · 5 bedrooms · 5 beds · 2 baths,Yasmin (MHO159) for 10 persons.,empty,$17 per night
7,4 guests,4 guests · 2 bedrooms · 2 beds · 3 baths,Spacious apartment accessible Mayrhofen,5.0,$177 per night
8,4 guests,4 guests · 2 bedrooms · 3 beds · 1 bath,"Apartment Bergglück, Bergglück 4",empty,$158 per night
9,12 guests,12 guests · 6 bedrooms · 15 beds · 2 shared baths,Quaint living in the Melcherhof,4.92,$160 per night


In [25]:
df.shape

(20, 5)